In [56]:
import glob
import zipfile 
import os
import xml.etree.ElementTree as ET
import collections
import csv
import random
from bs4 import BeautifulSoup
import progressbar
import numpy as np

In [82]:
paths_common = ['GESTION/NOJO',
                'GESTION/TETIER_R4',
                'GESTION/NOM_HTML',
                'DONNEES/IDENT/NOM',
                'DONNEES/IDENT/PRM',
                'DONNEES/IDENT/ADRESSE',
                'DONNEES/IDENT/CP',
                'DONNEES/IDENT/VILLE',
                'DONNEES/IDENT/MEL',
                'DONNEES/IDENT/TEL',
                'DONNEES/OBJET/OBJET_COMPLET']

give_me_the_tag = ['DONNEES/INFO',
                   'DONNEES/TYPE_ACTIVITE_ORG']

paths_take_all = ['DONNEES/CLASSES',
                  'DONNEES/DESCRIPTEURS',
                  'DONNEES/CARACTERISTIQUES']

jump_a_line = ['DONNEES/PROCEDURES/LOTS']

In [70]:
def path2tag(path):
    
    pavel = ''
    
    for i in reversed(path):
        if i != '/':
            pavel += i
        if i == '/':
            break
    return pavel[::-1]

In [69]:
path

'TELPMOC_TEJBO'

In [102]:
def xml_parser(paths_common, paths_take_all, file):
    pairs=[]
    
    for path in paths_common:
        element = file.find(path)
        if element is not None:
            if element.text is not None:
                pairs.append([element.tag, element.text])
            else:
                pairs.append([element.tag, ''])
        elif element is None:
            pairs.append([path2tag(path), ''])
    
    for path in give_me_the_tag:
        element = file.find(path)
        pavel = ''
        if element is not None:
            for child in element.getchildren():
                if child is not None:
                    pavel += child.tag
            pairs.append([path2tag(path), pavel])
        elif element is None:
            pairs.append([path2tag(path), ''])
            
        
         
    for path in paths_take_all:
        element = file.find(path)
        if element is not None:
            reslist = list(element.iter())
            result = ' '.join([element.text for element in reslist if element.text is not None])
            pairs.append([element.tag, result])
        
            
    return pairs 


In [73]:
def write_to_csv(big_pair_list, file_name):
    bar3 = pbar(len(big_pair_list))
    big=[]
    with open('{}.csv'.format(file_name), 'w') as file:
        spamwriter = csv.writer(file, delimiter=',', quotechar=' ',
                            quoting=csv.QUOTE_ALL)
        for pair_list in big_pair_list:
            bar3.update(big_pair_list.index(pair_list))
            
            #print(pair_list)
            tags = []
            text = []
            
            for pair in pair_list:
                tags.append(pair[0])
                text.append(pair[1])
            #spamwriter.writerow(tags)
            #spamwriter.writerow(text)
            big.append(tags)
            big.append(text)
        np.savetxt(file, np.asarray(big), delimiter=",", fmt='%s')
            

In [8]:
def pbar(maxlen):
    
    bar = progressbar.ProgressBar(maxval=maxlen, \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage(), ' (', progressbar.ETA(), ') '])
    
    bar.start()
    
    return bar

In [75]:
def write_years2csv(start, end):
    
    years = [start + i for i in range(end - start + 1)]
    
    for i in range(len(years)):
        paths = glob.glob('{}/mpc/*.xml'.format(years[i]))
        annonces = []
    
        bar = pbar(len(paths))
        
    
        for path in paths:
            bar.update(paths.index(path))
            file = open(path, encoding='ISO-8859-1')
            apple = file.read()
            
            apple = apple.replace('&apos;', "'")
            apple = apple.replace('&deg;', '°')
            apple = apple.replace('&quot;', '"')
            apple = apple.replace('\t', '')
            apple = apple.replace(',', '')
            apple = apple.replace('\n', '')
            
            tree = ET.fromstring(apple)
            a = tree.getchildren()
    
            annonces.append(a)
    
        p = []
        bar2 = pbar(len(annonces))
        for annonce in annonces:
            bar2.update(annonces.index(annonce))
            for j in range(len(annonce)):
                extracted_annonce = xml_parser(paths_common, paths_take_all, annonce[j])
                p.append(extracted_annonce)
        write_to_csv(p, years[i])

    

In [ ]:
write_years2csv(2005, 2005)

[===============================                       ]  59% (ETA:   0:02:28) 

In [81]:
paths = glob.glob('2005/mpa/*.xml')
file = open(random.choice(paths), encoding='ISO-8859-1')
apple = file.read()
apple = apple.replace('&apos;', "'")
apple = apple.replace('&deg;', '°')
apple = apple.replace('&quot;', '"')
tree = ET.fromstring(apple)
a = tree.getchildren()


In [83]:
classes = a[10].find(give_me_the_tag[1])

In [100]:
reslist = list(classes.iter())
result = ' '.join([element.text for element in reslist])

In [55]:
import numpy as np
a = np.asarray([["pipi", "caca, prout"], ["chiasse", ""]])
numpy.savetxt("foo.csv", a, delimiter=",", fmt='%s')
